In [1]:
from google.colab import drive
import os
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

import tensorflow as tf
from functools import partial
!pip install ray
from ray import tune
from ray.tune import CLIReporter
!pip install tensorboardX

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/divergence_hashing_2')

     |████████████████████████████████| 54.7 MB 36 kB/s 
     |████████████████████████████████| 72 kB 487 kB/s 
     |████████████████████████████████| 124 kB 4.2 MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWiAgBJSuWzmZtexMS-ERkOpitCfevApld-CpiQSih8yzXa-PO4K048
Mounted at /content/gdrive


In [2]:
seed_val = 42

np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [3]:
class SiameseDataset(Dataset):
    def __init__(self, sequence_pairs):
        self.len = len(sequence_pairs[0])
        self.original_sequences = sequence_pairs[0]
        self.mutated_sequences = sequence_pairs[1]
        
    def __getitem__(self, index):
        original_sequence = self.original_sequences[index]
        mutated_sequence = self.mutated_sequences[index]

        return {
            'original_sequence': original_sequence,
            'mutated_sequence': mutated_sequence
        }
    
    def __len__(self):
        return self.len

In [4]:
def makelen128(sequence):

    old_len = len(sequence)
    new_seq = []

    for i in range(0,128):
        index = i%old_len
        new_seq.append(sequence[index])
    
    new_seq = ''.join(map(str, new_seq))

    return new_seq

def has_homopolymer(seq):

    same_conseq = 0
    previous = None
    flag = False
    start_pos = -1
    end_pos = -1
    in_run = False
    position_list = []
    homopolymering = False

    for i in range(len(seq)):
        current = seq[i]
        if current is previous and current is not None:
            same_conseq += 1
            if same_conseq is 4:
                flag = True
                homopolymering = True
        if current is not previous or i is (len(seq)-1):
            same_conseq = 1
            if homopolymering is True:
                position_list.append([start_pos, i]) # end-exclusive
                homopolymering = False
            start_pos = i
        previous = current

    return flag, position_list


def produces_homopolymer(seq, position, base):

    old_flag, old_position_list = has_homopolymer(seq)
    copy_seq = seq[:]
    copy_seq[position] = base
    flag, position_list = has_homopolymer(copy_seq)
    if len(position_list) > len(old_position_list):
        return True
    return False

def get_CG_count(temp, primer_length):

    if primer_length < 14:
        CG_count = int((temp-primer_length*2)/2)

    elif primer_length == 18 and temp >= 43 and temp <= 72:
        if temp >= 43 and temp < 46:
            CG_count = 6
        elif temp >= 46 and temp < 48:
            CG_count = 7
        elif temp >= 48 and temp < 51:
            CG_count = 8
        elif temp >= 51 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 56:
            CG_count = 10
        elif temp >= 56 and temp < 59:
            CG_count = 11
        elif temp >= 59 and temp < 62:
            CG_count = 12
        elif temp >= 62 and temp < 65:
            CG_count = 13
        elif temp >= 65 and temp < 68:
            CG_count = 14
        elif temp >= 68 and temp < 71:
            CG_count = 15
        elif temp >= 71 and temp <= 72:
            CG_count = 16

    elif primer_length == 19 and temp >= 44 and temp <= 71:
        if temp >= 44 and temp < 46:
            CG_count = 6
        elif temp >= 46 and temp < 49:
            CG_count = 7
        elif temp >= 49 and temp < 51:
            CG_count = 8
        elif temp >= 51 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 57:
            CG_count = 10
        elif temp >= 57 and temp < 59:
            CG_count = 11
        elif temp >= 59 and temp < 62:
            CG_count = 12
        elif temp >= 62 and temp < 65:
            CG_count = 13
        elif temp >= 65 and temp < 68:
            CG_count = 14
        elif temp >= 68 and temp < 70:
            CG_count = 15
        elif temp >= 70 and temp <= 71:
            CG_count = 16

    elif primer_length == 20 and temp >= 45 and temp <= 70:
        if temp >= 45 and temp < 47:
            CG_count = 6
        elif temp >= 47 and temp < 49:
            CG_count = 7
        elif temp >= 49 and temp < 52:
            CG_count = 8
        elif temp >= 52 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 57:
            CG_count = 10
        elif temp >= 57 and temp < 59:
            CG_count = 11
        elif temp >= 59 and temp < 61:
            CG_count = 12
        elif temp >= 61 and temp < 64:
            CG_count = 13
        elif temp >= 64 and temp < 66:
            CG_count = 14
        elif temp >= 66 and temp < 69:
            CG_count = 15
        elif temp >= 69 and temp <= 70:
            CG_count = 16

    elif primer_length == 21 and temp >= 45 and temp <= 70:
        if temp >= 45 and temp < 47:
            CG_count = 6
        elif temp >= 47 and temp < 49:
            CG_count = 7
        elif temp >= 49 and temp < 52:
            CG_count = 8
        elif temp >= 52 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 56:
            CG_count = 10
        elif temp >= 56 and temp < 59:
            CG_count = 11
        elif temp >= 59 and temp < 61:
            CG_count = 12
        elif temp >= 61 and temp < 63:
            CG_count = 13
        elif temp >= 63 and temp < 66:
            CG_count = 14
        elif temp >= 66 and temp < 69:
            CG_count = 15
        elif temp >= 69 and temp <= 70:
            CG_count = 16

    elif primer_length == 22 and temp >= 46 and temp <= 69:
        if temp >= 46 and temp < 48:
            CG_count = 6
        elif temp >= 48 and temp < 50:
            CG_count = 7
        elif temp >= 50 and temp < 52:
            CG_count = 8
        elif temp >= 52 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 56:
            CG_count = 10
        elif temp >= 56 and temp < 58:
            CG_count = 11
        elif temp >= 58 and temp < 61:
            CG_count = 12
        elif temp >= 61 and temp < 63:
            CG_count = 13
        elif temp >= 63 and temp < 65:
            CG_count = 14
        elif temp >= 65 and temp < 68:
            CG_count = 15
        elif temp >= 68 and temp <= 69:
            CG_count = 16

    elif primer_length == 23 and temp >= 46 and temp <= 68:
        if temp >= 46 and temp < 48:
            CG_count = 6
        elif temp >= 48 and temp < 50:
            CG_count = 7
        elif temp >= 50 and temp < 51:
            CG_count = 8
        elif temp >= 51 and temp < 53:
            CG_count = 9
        elif temp >= 53 and temp < 55:
            CG_count = 10
        elif temp >= 55 and temp < 57:
            CG_count = 11
        elif temp >= 57 and temp < 61:
            CG_count = 12
        elif temp >= 61 and temp < 63:
            CG_count = 13
        elif temp >= 63 and temp < 65:
            CG_count = 14
        elif temp >= 65 and temp < 67:
            CG_count = 15
        elif temp >= 67 and temp <= 68:
            CG_count = 16

    elif primer_length == 24 and temp >= 47 and temp <= 68:
        if temp >= 47 and temp < 49:
            CG_count = 6
        elif temp >= 49 and temp < 51:
            CG_count = 7
        elif temp >= 51 and temp < 52:
            CG_count = 8
        elif temp >= 52 and temp < 54:
            CG_count = 9
        elif temp >= 54 and temp < 56:
            CG_count = 10
        elif temp >= 56 and temp < 58:
            CG_count = 11
        elif temp >= 58 and temp < 60:
            CG_count = 12
        elif temp >= 60 and temp < 62:
            CG_count = 13
        elif temp >= 62 and temp < 65:
            CG_count = 14
        elif temp >= 65 and temp < 67:
            CG_count = 15
        elif temp >= 67 and temp <= 68:
            CG_count = 16

    else:
        CG_count = int(round(0.865*((primer_length*(temp - 64.9))/41 + 16.4)))
    
    CG_count = max(0, CG_count)
    CG_count = min(CG_count, primer_length)
    return CG_count


def seq_to_temp(seq):

    primer_length = len(seq)
    CG_count = 0
    dict = {
        'A': 0,
        'T': 0,
        'C': 1,
        'G': 1
    }       

    for base in seq:
        CG_count += dict[base] 

    if primer_length < 14:
        temp = CG_count*4 + (primer_length-CG_count)*2

    elif primer_length == 18 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 43.9
        elif CG_count == 7:
            temp = 46.3
        elif CG_count == 8:
            temp = 49.4
        elif CG_count == 9:
            temp = 51.7
        elif CG_count == 10:
            temp = 54.4
        elif CG_count == 11:
            temp = 57.2
        elif CG_count == 12:
            temp = 60.2
        elif CG_count == 13:
            temp = 63.2
        elif CG_count == 14:
            temp = 66.4
        elif CG_count == 15:
            temp = 69.0
        elif CG_count == 16:
            temp = 71.7

    elif primer_length == 19 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 44.6
        elif CG_count == 7:
            temp = 47.0
        elif CG_count == 8:
            temp = 49.5
        elif CG_count == 9:
            temp = 52.1
        elif CG_count == 10:
            temp = 54.7
        elif CG_count == 11:
            temp = 57.3
        elif CG_count == 12:
            temp = 60.0
        elif CG_count == 13:
            temp = 62.7
        elif CG_count == 14:
            temp = 65.7
        elif CG_count == 15:
            temp = 68.5
        elif CG_count == 16:
            temp = 70.8

    elif primer_length == 20 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 45.2
        elif CG_count == 7:
            temp = 47.5
        elif CG_count == 8:
            temp = 50.2
        elif CG_count == 9:
            temp = 52.4
        elif CG_count == 10:
            temp = 54.5
        elif CG_count == 11:
            temp = 57.0
        elif CG_count == 12:
            temp = 59.7
        elif CG_count == 13:
            temp = 62.1
        elif CG_count == 14:
            temp = 64.6
        elif CG_count == 15:
            temp = 67.4
        elif CG_count == 16:
            temp = 70.1

    elif primer_length == 21 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 45.4
        elif CG_count == 7:
            temp = 47.8
        elif CG_count == 8:
            temp = 50.0
        elif CG_count == 9:
            temp = 52.2
        elif CG_count == 10:
            temp = 54.5
        elif CG_count == 11:
            temp = 57.0
        elif CG_count == 12:
            temp = 59.3
        elif CG_count == 13:
            temp = 61.8
        elif CG_count == 14:
            temp = 64.3
        elif CG_count == 15:
            temp = 66.9
        elif CG_count == 16:
            temp = 69.6

    elif primer_length == 22 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 46.1
        elif CG_count == 7:
            temp = 48.1
        elif CG_count == 8:
            temp = 50.3
        elif CG_count == 9:
            temp = 52.5
        elif CG_count == 10:
            temp = 54.7
        elif CG_count == 11:
            temp = 56.9
        elif CG_count == 12:
            temp = 59.1
        elif CG_count == 13:
            temp = 61.3
        elif CG_count == 14:
            temp = 63.6
        elif CG_count == 15:
            temp = 66.0
        elif CG_count == 16:
            temp = 68.3

    elif primer_length == 23 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 46.6
        elif CG_count == 7:
            temp = 48.7
        elif CG_count == 8:
            temp = 50.8
        elif CG_count == 9:
            temp = 52.7
        elif CG_count == 10:
            temp = 54.8
        elif CG_count == 11:
            temp = 56.9
        elif CG_count == 12:
            temp = 59.0
        elif CG_count == 13:
            temp = 61.2
        elif CG_count == 14:
            temp = 63.4
        elif CG_count == 15:
            temp = 65.5
        elif CG_count == 16:
            temp = 67.8

    elif primer_length == 24 and CG_count >= 6 and CG_count <= 16:
        if CG_count == 6:
            temp = 47.2
        elif CG_count == 7:
            temp = 49.1
        elif CG_count == 8:
            temp = 51.0
        elif CG_count == 9:
            temp = 53.0
        elif CG_count == 10:
            temp = 54.9
        elif CG_count == 11:
            temp = 56.9
        elif CG_count == 12:
            temp = 58.9
        elif CG_count == 13:
            temp = 60.9
        elif CG_count == 14:
            temp = 62.9
        elif CG_count == 15:
            temp = 65.1
        elif CG_count == 16:
            temp = 67.2
    else:
        temp = 64.9 + 41*(CG_count-16.4)/primer_length
        
    return temp

def logits_to_primer(logits, temp, primer_length):

    CG_count = get_CG_count(temp, primer_length)

    logits = nn.functional.softmax(logits, dim = 1)
    logits = torch.transpose(logits, 2, 1) #returns 6,128,4
    logits = logits[:, :primer_length, :]
    pred = torch.argmax(logits, 2)
    pred = pred.tolist() #returns list of list (6, 128)
    size = list(logits.size())
    sequence_list = []
    int2base = {0: "A", 1: "T", 2: "C", 3: "G"}

    for seq in range(size[0]):
        CG_some_data = []
        AT_some_data = []
        sequence = [None]*size[1]
        base_sequence = [None]*size[1]
        for position in range(size[1]):
            data0 = [2, logits[seq][position][2], position]
            data1 = [3, logits[seq][position][3], position]
            CG_some_data.append(data0)
            CG_some_data.append(data1)

            data0 = [0, logits[seq][position][0], position]
            data1 = [1, logits[seq][position][1], position]
            AT_some_data.append(data0)
            AT_some_data.append(data1)
        CG_sorted_some_data = sorted(range(len(CG_some_data)), key = lambda k: CG_some_data[k][1], reverse=True)
        AT_sorted_some_data = sorted(range(len(CG_some_data)), key = lambda k: AT_some_data[k][1], reverse=True)

        # try to place CGs without causing homopolymers
        placed_CG = 0
        for i in range(len(CG_sorted_some_data)):
            if placed_CG < CG_count:
                index = CG_sorted_some_data[i]
                base = CG_some_data[index][0]
                position = CG_some_data[index][2]
                if sequence[position] is None and not produces_homopolymer(sequence, position, base):
                    sequence[position] = base
                    placed_CG+=1

        # check for 5' and 3' GC clamps (since extension happens from 3' of this primer and the complementary primer)
        five_prime_GC_clamp = 0
        three_prime_GC_clamp = 0
        mid_GC_count = 0

        for i in range(5):
            if sequence[i] is not None:
                five_prime_GC_clamp+= 1
        
        for i in range(len(sequence)-1, len(sequence)-6, -1):
            if sequence[i] is not None:
                three_prime_GC_clamp+= 1

        for i in range(5, len(sequence)-1):
            if sequence[i] is not None:
                mid_GC_count+=1

        CG_to_add = 0
        # remove GCs in excess of 3 in five_prime_GC_clamp
        for i in range(len(CG_sorted_some_data)-1, -1, -1):
            if five_prime_GC_clamp > 3:
                index = CG_sorted_some_data[i]
                base = CG_some_data[index][0]
                position = CG_some_data[index][2]
                if position < 5 and sequence[position] is not None:
                    sequence[position] = None
                    CG_to_add+=1
                    five_prime_GC_clamp-=1

        # remove GCs in excess of 3 in three_prime_GC_clamp
        for i in range(len(CG_sorted_some_data)-1, -1, -1):
            if three_prime_GC_clamp > 3:
                index = CG_sorted_some_data[i]
                base = CG_some_data[index][0]
                position = CG_some_data[index][2]
                if position > primer_length-6 and sequence[position] is not None:
                    sequence[position] = None
                    CG_to_add+=1
                    three_prime_GC_clamp-=1

        # add back GCs elsewhere
        pointer = 0 # those before were already placed or not placed as they cause homopolymers
        while CG_to_add > 0 and pointer is not len(CG_sorted_some_data):
            index = CG_sorted_some_data[pointer]
            base = CG_some_data[index][0]
            position = CG_some_data[index][2]
            if sequence[position] is None and not produces_homopolymer(sequence, position, base):
                if position > 4 and position < primer_length-5:
                    sequence[position] = base
                    CG_to_add-=1                
                    mid_GC_count+=1
                elif position < 5 and five_prime_GC_clamp < 3:
                    sequence[position] = base
                    CG_to_add-=1
                    five_prime_GC_clamp+=1
                elif position > primer_length-6 and three_prime_GC_clamp < 3:
                    sequence[position] = base
                    CG_to_add-=1
                    three_prime_GC_clamp+=1
            pointer+=1

        # if we still were not able to put back the GCs elsewhere, then put it back anyway
        #(even if this results in more than 3 CGs in the ends)
        #(e.g. if a ridiculously high temp that causes the whole seq to be CGs was set)
        pointer = 0
        while CG_to_add > 0 and pointer is not len(CG_sorted_some_data):
            index = CG_sorted_some_data[pointer]
            base = CG_some_data[index][0]
            position = CG_some_data[index][2]
            if sequence[position] is None and not produces_homopolymer(sequence, position, base):
                sequence[position] = base
                CG_to_add-=1
                if position > 4 and position < primer_length-5:                
                    mid_GC_count+=1
                elif position < 5 and five_prime_GC_clamp < 3:
                    five_prime_GC_clamp+=1
                elif position > primer_length-6 and three_prime_GC_clamp < 3:
                    three_prime_GC_clamp+=1
            pointer+=1

        # if 5' GC clamp not present and there is at least one GC in the flanked region or more than one in 3' region
        # (to remove after for temp considerations), add the 5' GC clamp
        if five_prime_GC_clamp is 0 and (mid_GC_count is not 0 or three_prime_GC_clamp > 1):
            for i in range(CG_count, len(CG_sorted_some_data)):
                index = CG_sorted_some_data[i]
                position = CG_some_data[index][2]
                base = CG_some_data[index][0]
                if position < 5 and not produces_homopolymer(sequence, position, base):
                    sequence[position] = base
                    five_prime_GC_clamp+=1
                    break

            if three_prime_GC_clamp > 1:
                # remove a GC from the flanked region or the 3' region by going from the least probable CG
                for i in range(len(CG_sorted_some_data)-1, -1, -1):
                    index = CG_sorted_some_data[i]
                    position = CG_some_data[index][2]
                    base = CG_some_data[index][0]
                    if position > 4 and position < primer_length-5 and sequence[position] is not None:
                        sequence[position] = None
                        mid_GC_count-=1
                        break
                    elif position > primer_length-6 and sequence[position] is not None:
                        sequence[position] = None
                        three_prime_GC_clamp-=1
                        break                        

            else:
                # remove a GC from the flanked region
                for i in range(len(CG_sorted_some_data)-1, -1, -1):
                    index = CG_sorted_some_data[pointer]
                    position = CG_some_data[index][2]
                    base = CG_some_data[index][0]
                    if position > 4 and position < primer_length-5 and sequence[position] is not None:
                        sequence[position] = None
                        mid_GC_count-=1
                        break                      

        # if 3' GC clamp not present and there are at least two GCs elsewhere (one in 5' region one in flanking) add the 3' GC clamp
        if three_prime_GC_clamp is 0 and (mid_GC_count is not 0 or five_prime_GC_clamp > 1):
            for i in range(CG_count, primer_length):
                index = CG_sorted_some_data[i]
                position = CG_some_data[index][2]
                base = CG_some_data[index][0]
                if position > primer_length-6 and not produces_homopolymer(sequence, position, base):
                    sequence[position] = base
                    three_prime_GC_clamp+=1
                    break

            if five_prime_GC_clamp > 1:
                # remove a GC from the flanked region or the 3' region by going from the least probable CG
                for i in range(len(CG_sorted_some_data)-1, - 1, -1):
                    index = CG_sorted_some_data[i]
                    position = CG_some_data[index][2]
                    base = CG_some_data[index][0]
                    if position > 4 and position < primer_length-5 and sequence[position] is not None:
                        sequence[position] = None
                        mid_GC_count-=1
                        break
                    elif position < 5 and sequence[position] is not None:
                        sequence[position] = None
                        five_prime_GC_clamp-=1
                        break                        

            else:
                for i in range(len(CG_sorted_some_data)-1, -1, -1):
                    index = CG_sorted_some_data[i]
                    position = CG_some_data[index][2]
                    base = CG_some_data[index][0]
                    # if it is in the flanked region and has yet to be removed
                    if position > 4 and position < primer_length-5 and sequence[position] is not None:
                        sequence[position] = None
                        mid_GC_count-=1
                        break        

        for i in range(len(AT_sorted_some_data)):
            index = AT_sorted_some_data[i]
            position = AT_some_data[index][2]
            base = AT_some_data[index][0]
            if sequence[position] is None and not produces_homopolymer(sequence, position, base):
                sequence[position] = base

        for i in range(len(sequence)):
            base_sequence[i] = int2base[int(sequence[i])]
        base_sequence = ''.join(map(str, base_sequence))
        sequence_list.append(base_sequence)
    return sequence_list

In [5]:
class SiameseModel(nn.Module):
    
    def __init__(self, single_model):

        super().__init__()
        self.single_model = single_model

    def forward(self, input_0, input_1):

        logits_0, original_0, sequence_list_0 = self.single_model(input_0)

        logits_1, original_1, sequence_list_1 = self.single_model(input_1)

        loss_fct = nn.KLDivLoss()

        b_original_mismatches = []
        b_mismatches = []
        b_loss = 0
        total_mismatches = 0
            
        size = len(input_0)
        for x in range(size):
            mismatches = sum(1 for a, b in zip(sequence_list_0[x], sequence_list_1[x]) if a != b)
            total_mismatches += mismatches
            b_mismatches.append(mismatches)

            original_mismatches = sum(1 for a, b in zip(original_0[x], original_1[x]) if a != b)
            b_original_mismatches.append(original_mismatches)

            scaled_logits_0 = torch.mul(logits_0[x],10)
            scaled_logits_1 = torch.mul(logits_1[x],10)
            loss = -loss_fct(torch.nn.functional.log_softmax(logits_0,dim=0), torch.nn.functional.softmax(logits_1,dim=0))
            b_loss += loss

        average_mismatches = total_mismatches/size

        if b_loss < -100:
            print(logits_0)
            raise ValueError()
        return b_loss, average_mismatches, (original_0, original_1), b_original_mismatches, (sequence_list_0, sequence_list_1), b_mismatches, (logits_0, logits_1)

In [6]:
def seq_to_tensor(input):
    array = []
    base_to_int = {
        'A': 0,
        'T': 1,
        'C': 2,
        'G': 3
    }
    for i in input:
        array.append(base_to_int[i])
    rows = np.max(array) + 1
    one_hot = np.eye(rows)[array]
    one_hot_transpose = np.transpose(one_hot)
    return one_hot_transpose

def multiple_seq_to_tensor(input_list):
    output_list = []
    for i in range(len(input_list)):
        array = seq_to_tensor(input_list[i])
        output_list.append(array.tolist())
    tensor = torch.tensor(output_list)
    return tensor

In [7]:
class Net(nn.Module):

    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        super().__init__()
        
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.fc3 = nn.Linear(hidden2_size, hidden3_size)
        self.fc4 = nn.Linear(hidden3_size, output_size)

        nn.init.kaiming_uniform_(self.fc1.weight)
        nn.init.kaiming_uniform_(self.fc2.weight)
        nn.init.kaiming_uniform_(self.fc3.weight)
        nn.init.kaiming_uniform_(self.fc4.weight)
    
    def logits_to_seq(self, logits):

        logits = torch.transpose(logits, 2, 1) #returns 6,128,4

        # return list of list
        pred = torch.argmax(logits, 2)
        pred = pred.tolist()

        size = list(logits.size())
        int_to_base = {
          0: "A",
          1: "T",
          2: "C",
          3: "G"
        }
        sequence_list = [None]*len(pred)

        for seq in range(size[0]):
          for base in range(size[1]):
              pred[seq][base] = int_to_base[pred[seq][base]]
          sequence = ''.join(map(str, pred[seq]))
          sequence_list[seq] = sequence
        return sequence_list

    def forward(self, x):
        seq_in = self.logits_to_seq(x)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = self.fc4(x)
        seq_out = self.logits_to_seq(x)
        return x, seq_in, seq_out

In [8]:
config="tuned"
n_gpu = 1
root_dir = ''

class Regressor():

    def __init__(self, nb_epoch=1000, config=None):

        super(Regressor, self).__init__()

        if config is not None:
            self.nb_epoch = config["num_epochs"]
            self.batch_size = config["batch_size"]
            self.learning_rate = config["learning_rate"]
            self.hidden1_size = config["hidden1_size"]
            self.hidden2_size = config["hidden2_size"]
            self.hidden3_size = config["hidden3_size"]
            self.is_tuning = True

        else:
            self.nb_epoch = nb_epoch
            self.batch_size = 32
            self.learning_rate = 0.0352048104552604
            self.hidden1_size = 20
            self.hidden2_size = 100
            self.hidden3_size = 180
            self.is_tuning = False

        self.input_size = 128
        self.output_size = 128

        self.net = Net(self.input_size, self.hidden1_size, self.hidden2_size, self.hidden3_size, self.output_size)
        self.siamese_net = SiameseModel(self.net)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.siamese_net.to(device)
        print(device)

        return

    def one_hot_seq(input):
        array = []
        base_to_int = {
            'A': 0,
            'T': 1,
            'C': 2,
            'G': 3
        }
        for i in input:
            array.append(base_to_int[i])
        rows = np.max(array) + 1
        one_hot = np.eye(rows)[array]
        return one_hot


    def multiple_seq_to_tensor(input_list):
        output_list = []
        for i in range(len(input_list)):
            array = one_hot_seq(input_list[i])
            output_list.append(array.tolist())
        tensor = torch.tensor(output_list)
        return tensor


    def _preprocessor(self, train_sequence_0, train_sequence_1, training=False):

        processed_original = multiple_seq_to_tensor(train_sequence_0)
        processed_mutated = multiple_seq_to_tensor(train_sequence_1)

        return processed_original, processed_mutated

    def test_loop(self, epoch, validation_dataloader, stats=False):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = self.siamese_net
        model.eval()

        original_seq = []
        mutated_seq = []
        original_hashed = []
        mutated_hashed = []
        ori_mismatches = []
        mismatches = []
    
        for step, batch in enumerate(tqdm(validation_dataloader, position=0, leave=True)):

            b_original_seq, b_mutated_seq = batch
            b_original_seq = b_original_seq.to(device)
            b_mutated_seq = b_mutated_seq.to(device)

            with torch.no_grad():
                loss, average_mismatches, (in_0, in_1), b_original_mismatches, (pred_0, pred_1), b_mismatches, (logits_0, logits_1) = model.forward(b_original_seq, b_mutated_seq)
        
            for i in range(len(b_original_seq)):
                original_seq.append(in_0[i])
                mutated_seq.append(in_1[i])
                original_hashed.append(pred_0[i])
                mutated_hashed.append(pred_1[i])
                ori_mismatches.append(b_original_mismatches[i])
                mismatches.append(b_mismatches[i])

        if stats:
            test_info = {'original_seq': original_seq,
                'mutated_seq': mutated_seq,
                'original_hashed': original_hashed,
                'mutated_hashed': mutated_hashed,
                'original_mismatches': ori_mismatches,
                'hashed_mismatches': mismatches}
            test_info_df = pd.DataFrame.from_dict(test_info)
            test_info_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/test_info_df_epoch_{}.csv'.format(config, epoch))

        average_mismatches = sum(mismatches)/len(mismatches)
        print("Test: On average, there are {} mismatches".format(average_mismatches))
        return average_mismatches


    def train_loop(self, epoch, train_dataloader, optimizer, train_loss_set, stats=False):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        model = self.siamese_net

        original_seq = []
        mutated_seq = []
        original_hashed = []
        mutated_hashed = []
        original_mismatches = []
        mismatches = []

        for step, batch in enumerate(tqdm(train_dataloader, position=0, leave=True)):

            model.to(device)
            model.train()

            b_original_seq, b_mutated_seq = batch
            b_original_seq = b_original_seq.to(device)
            b_mutated_seq = b_mutated_seq.to(device)
        
            optimizer.zero_grad()
            loss, average_mismatches, (in_0, in_1), ori_b_mismatches, (pred_0, pred_1), b_mismatches, (logits_0, logits_1) = model.forward(b_original_seq, b_mutated_seq)
        
            if n_gpu > 1:
                loss = loss.mean()
            train_loss_set.append(loss.item())
            loss.backward()
            optimizer.step()

            for i in range(len(b_original_seq)):
                original_seq.append(in_0[i])
                mutated_seq.append(in_1[i])
                original_hashed.append(pred_0[i])
                mutated_hashed.append(pred_1[i])
                original_mismatches.append(ori_b_mismatches[i])
                mismatches.append(b_mismatches[i])

        if stats:
            train_info = {'original_seq': original_seq,
                    'mutated_seq': mutated_seq,
                    'original_hashed': original_hashed,
                    'mutated_hashed': mutated_hashed,
                    'original_mismatches': original_mismatches,
                    'mismatches': mismatches}
            train_info_df = pd.DataFrame.from_dict(train_info)
            train_info_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/train_info_df_epoch_{}.csv'.format(config, epoch))
        
        average_mismatches = sum(mismatches)/len(mismatches)
        average_loss = sum(train_loss_set)/len(train_loss_set)
        print("Train: On average, there are {} mismatches for epoch {}".format(average_mismatches, epoch))
        print("Train: On average, the training loss is {} for epoch {}".format(average_loss, epoch))
        return average_mismatches


    def fit(self, train_sequence_0, train_sequence_1):

        train_sequence_0, validation_sequence_0, train_sequence_1, validation_sequence_1 = train_test_split(train_sequence_0, train_sequence_1,
                                                                                    random_state=42, test_size=0.1)
        train_sequence_0, train_sequence_1 = self._preprocessor(train_sequence_0, train_sequence_1, training=True)
        validation_sequence_0, validation_sequence_1 = self._preprocessor(validation_sequence_0, validation_sequence_1, training=True)  # Do not forget

        train_data = torch.utils.data.TensorDataset(train_sequence_0, train_sequence_1)
        train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=self.batch_size)

        validation_data = torch.utils.data.TensorDataset(validation_sequence_0, validation_sequence_1)
        validation_dataloader = torch.utils.data.DataLoader(validation_data, batch_size=self.batch_size)

        optimizer = optim.SGD(self.net.parameters(), lr=self.learning_rate)
        
        test_average_mismatches = []

        train_loss_set = []
        train_average_mismatches = []

        last_validation_mismatch = 0

        for epoch in range(self.nb_epoch):
            print("Epoch: {}".format(epoch))
            stats_on = True
            train_average_mismatch = self.train_loop(epoch, train_dataloader, optimizer, train_loss_set, stats=stats_on)
            train_average_mismatches.append(train_average_mismatch)
            test_average_mismatch = self.test_loop(epoch, validation_dataloader, stats=stats_on)
            test_average_mismatches.append(test_average_mismatch)
            last_validation_mismatch = test_average_mismatch

        if self.is_tuning:
            tune.report(mismatches=last_validation_mismatch)

        train_average_mismatches_df = pd.DataFrame(data=train_average_mismatches)
        train_average_mismatches_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/train_average_mismatches.csv'.format(config))

        test_average_mismatches_df = pd.DataFrame(data=test_average_mismatches)
        test_average_mismatches_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/test_average_mismatches.csv'.format(config))

        train_loss_df = pd.DataFrame(data=train_loss_set)
        train_loss_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/training_losses.csv'.format(config))

        return self


    def loop_helper(self, count, sequence_0, sequence_1, called_by_score=False):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        sequence_0, sequence_1 = self._preprocessor(sequence_0, sequence_1, training=False)
        data = torch.utils.data.TensorDataset(sequence_0, sequence_1)
        dataloader = torch.utils.data.DataLoader(data, batch_size=self.batch_size)

        model = self.siamese_net
        model = model.to(device)
        model.eval()

        original_seq = []
        mutated_seq = []
        original_hashed = []
        mutated_hashed = []
        original_mismatches = []
        mismatches = []
        logits_0 = []
        logits_1 = []
    
        for step, batch in enumerate(tqdm(dataloader, position=0, leave=True)):

            b_original_seq, b_mutated_seq = batch
            b_original_seq = b_original_seq.to(device)
            b_mutated_seq = b_mutated_seq.to(device)

            with torch.no_grad():
                loss, average_mismatches, (in_0, in_1), ori_b_mismatches, (pred_0, pred_1), b_mismatches, (logit_0, logit_1) = model.forward(b_original_seq, b_mutated_seq)
        
            logit_0 = logit_0.tolist()
            logit_1 = logit_1.tolist()

            for i in range(len(b_original_seq)):
                original_seq.append(in_0[i])
                mutated_seq.append(in_1[i])
                original_hashed.append(pred_0[i])
                mutated_hashed.append(pred_1[i])
                original_mismatches.append(ori_b_mismatches[i])
                mismatches.append(b_mismatches[i])
                logits_0.append(logit_0[i])
                logits_1.append(logit_1[i])

        logits_0 = torch.FloatTensor(logits_0)
        logits_1 = torch.FloatTensor(logits_1)

        hash_info = {'original_seq': original_seq,
                 'mutated_seq': mutated_seq,
                 'original_hashed': original_hashed,
                 'mutated_hashed': mutated_hashed,
                 'original_mismatches': original_mismatches,
                 'mismatches': mismatches}
        hash_info_df = pd.DataFrame.from_dict(hash_info)
        hash_info_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/loop{}_info.csv'.format(config, count))

        average_mismatches = sum(mismatches)/len(mismatches)
        print("Loop: On average, there are {} mismatches".format(average_mismatches))
        return average_mismatches, original_hashed, mutated_hashed, logits_0, logits_1


    def loop(self, no_of_loops, sequence_0, sequence_1):

        in_0 = sequence_0
        in_1 = sequence_1

        average_mismatches = []

        for loop in range(no_of_loops):

            average_mismatch, in_0, in_1, logits_0, logits_1 = self.loop_helper(loop, in_0, in_1)
            average_mismatches.append(average_mismatch)

        average_mismatches_df = pd.DataFrame(data=average_mismatches)
        average_mismatches_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/loop_average_mismatches.csv'.format(config))

        return in_0, in_1


    def evaluate_loop(self, no_of_loops, sequence_0, sequence_1, temp, primer_length, subconfig):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        in_0, in_1 = sequence_0, sequence_1

        average_mismatches = []

        for loop in range(no_of_loops):

            average_mismatch, in_0, in_1, logits_0, logits_1 = self.loop_helper(loop, in_0, in_1)
            average_mismatches.append(average_mismatch)
      
        out_0 = logits_to_primer(logits_0, temp, primer_length)
        out_1 = logits_to_primer(logits_1, temp, primer_length)
        
        original_mismatches = []
        hashed_mismatches = []
        actual_temp_0 = []
        actual_temp_1 = []

        size = len(sequence_0)
        for x in range(size):
            original_mismatch = sum(1 for a, b in zip(sequence_0[x], sequence_1[x]) if a != b)
            original_mismatches.append(original_mismatch)

            temp_0 = seq_to_temp(out_0[x])
            actual_temp_0.append(temp_0)

            temp_1 = seq_to_temp(out_1[x])
            actual_temp_1.append(temp_1)            

            hashed_mismatch = sum(1 for a, b in zip(out_0[x], out_1[x]) if a != b)
            hashed_mismatches.append(hashed_mismatch)
        
        loop_info = {'original_seq': sequence_0,
                 'mutated_seq': sequence_1,
                 'original_hashed': out_0,
                 'mutated_hashed': out_1,
                 'temp_original_hashed': actual_temp_0,
                 'temp_mutated_hashed': actual_temp_1,
                 'original_mismatches': original_mismatches,
                 'hashed_mismatches': hashed_mismatches}
        loop_info_df = pd.DataFrame(loop_info)
        loop_info_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/{}_overall_loop_info.csv'.format(config, subconfig))

        average_mismatches_df = pd.DataFrame(data=average_mismatches)
        average_mismatches_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/{}_loop_average_mismatches.csv'.format(config, subconfig))

        average_mismatches = sum(original_mismatches)/len(original_mismatches)
        print("Predict: On average, there are {} mismatches".format(average_mismatches))

        average_temp_0 = sum(actual_temp_0)/len(actual_temp_0)
        average_temp_1 = sum(actual_temp_1)/len(actual_temp_1)
        print("Predict: On average, the melting temperatures are {} and {}".format(average_temp_0, average_temp_1))


    def loop_helper_no_stats(self, count, sequence_0, sequence_1, called_by_score=False):

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        sequence_0, sequence_1 = self._preprocessor(sequence_0, sequence_1, training=False)
        data = torch.utils.data.TensorDataset(sequence_0, sequence_1)
        dataloader = torch.utils.data.DataLoader(data, batch_size=self.batch_size)

        model = self.siamese_net
        model = model.to(device)
        model.eval()

        logits_0 = []
        logits_1 = []
        original_hashed = []
        mutated_hashed = []
    
        for step, batch in enumerate(tqdm(dataloader, position=0, leave=True)):

            b_original_seq, b_mutated_seq = batch
            b_original_seq = b_original_seq.to(device)
            b_mutated_seq = b_mutated_seq.to(device)

            with torch.no_grad():
                loss, average_mismatches, (in_0, in_1), ori_b_mismatches, (pred_0, pred_1), b_mismatches, (logit_0, logit_1) = model.forward(b_original_seq, b_mutated_seq)
        
            logit_0 = logit_0.tolist()
            logit_1 = logit_1.tolist()

            for i in range(len(b_original_seq)):
                logits_0.append(logit_0[i])
                logits_1.append(logit_1[i])
                original_hashed.append(pred_0[i])
                mutated_hashed.append(pred_1[i])

        logits_0 = torch.FloatTensor(logits_0)
        logits_1 = torch.FloatTensor(logits_1)

        return original_hashed, mutated_hashed, logits_0, logits_1


    def hash(self, file, no_of_loops, temp, primer_length):
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        df = pd.read_csv(file)
        sequences = list(df['original_seq'])

        for i in range(len(sequences)):
            sequences[i] = makelen128(sequences[i])

        sequence = sequences
        temps = []

        for loop in range(no_of_loops):

            sequence, sequence, logits, logits = self.loop_helper_no_stats(loop, sequence, sequence)
      
        seq_out = logits_to_primer(logits, temp, primer_length)

        for x in range(len(seq_out)):

            temp = seq_to_temp(seq_out[x])
            temps.append(temp)
        
        hash_info = {'original_seq': sequences,
                 'hashed_seq': seq_out,
                 'temp_hashed': temps}
        hash_info_df = pd.DataFrame(hash_info)
        hash_info_df.to_csv('/content/gdrive/My Drive/divergence_hashing_2/{}/hashed.csv'.format(config))


    def hash_one_seq(self, seq, no_of_loops, temp, primer_length):
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        seq = makelen128(seq)
        sequence = [seq]

        for loop in range(no_of_loops):

            sequence, sequence, logits, logits = self.loop_helper_no_stats(loop, sequence, sequence)
      
        seq_out = logits_to_primer(logits, temp, primer_length)

        temp = seq_to_temp(seq_out[0])

        print("The hashed sequence of melting temp {} is {}".format(temp, seq_out))

        return seq_out, temp

def save_regressor(trained_model):
    """
    Utility function to save the trained regressor model in part2_model.pickle.
    """
    # If you alter this, make sure it works in tandem with load_regressor
    with open(root_dir + config + 'model.pickle', 'wb') as target:
        pickle.dump(trained_model, target)
    print("\nSaved model in model.pickle\n")


def load_regressor(config):
    with open(root_dir + config + 'model.pickle', 'rb') as target:
        trained_model = pickle.load(target)
    print("\nLoaded model in model.pickle\n")
    return trained_model

def RayTuneHelper(x, y, config):
    model = Regressor(config=config)
    model.fit(x, y)

def RegressorHyperParameterSearch(x, y):

    tune_config = {
        "num_epochs": tune.grid_search([10, 100]),
        "batch_size": tune.grid_search([6, 32]),
        "learning_rate": tune.loguniform(1e-6, 1e-1),
        "hidden1_size": tune.grid_search([20, 100, 180]),
        "hidden2_size": tune.grid_search([20, 100, 180]),
        "hidden3_size": tune.grid_search([20, 100, 180])
    }

    reporter = CLIReporter(
        parameter_columns=["num_epochs", "batch_size", "learning_rate", "hidden1_size", "hidden2_size", "hidden3_size"],
        metric_columns=["mismatches"],
        print_intermediate_tables=True
    )

    result = tune.run(
        partial(RayTuneHelper, x, y),
        config=tune_config,
        progress_reporter=reporter
    )

    best_trial = result.get_best_trial(metric="mismatches", mode="max")
    print("Best trial config: {}".format(best_trial.config))

    return best_trial.config

In [11]:
def example_main():

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    root_dir = ''
    os.makedirs(config, exist_ok=True)
    df = pd.read_csv("2500.csv")
    #df = pd.read_csv("100thousand_upto1.csv")
    sequence_pairs = [list(df['seq1']),list(df['seq2'])]

    train_sequence_0, test_sequence_0, train_sequence_1, test_sequence_1 = train_test_split(sequence_pairs[0], sequence_pairs[1], test_size=0.10, shuffle=True, random_state=1)

    # Uncomment for hyperparameter search
    #RegressorHyperParameterSearch(train_sequence_0, train_sequence_1)

    regressor = Regressor(nb_epoch=10)
    regressor.fit(train_sequence_0, train_sequence_1)
    regressor.evaluate_loop(10, test_sequence_0, test_sequence_1, 60, 20, "test")

    save_regressor(regressor)

if __name__ == "__main__":
    example_main()

cuda
Epoch: 0


  0%|          | 0/64 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
100%|██████████| 64/64 [00:01<00:00, 33.26it/s]


Train: On average, there are 12.310123456790123 mismatches for epoch 0
Train: On average, the training loss is -0.004008214866189519 for epoch 0


100%|██████████| 8/8 [00:00<00:00, 68.53it/s]


Test: On average, there are 11.724444444444444 mismatches
Epoch: 1


...

Loop: On average, there are 95.388 mismatches


100%|██████████| 8/8 [00:00<00:00, 60.40it/s]


Loop: On average, there are 95.428 mismatches


100%|██████████| 8/8 [00:00<00:00, 60.75it/s]


Loop: On average, there are 95.592 mismatches
Predict: On average, there are 1.0 mismatches
Predict: On average, the melting temperatures are 59.70000000000025 and 59.70000000000025

Saved model in model.pickle



In [13]:
# the following shows how to use the hashing function after having trained a model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_dir = ''
config = "mismatch_analysis"
os.makedirs(config, exist_ok=True)

regressor = load_regressor("tuned")
'''
one_df = pd.read_csv("one.csv")

one_0, one_1 = list(one_df['seq1']),list(one_df['seq2'])
regressor.evaluate_loop(10, one_0, one_1, 62, 20, "one")

two_df = pd.read_csv("two.csv")
two_0, two_1 = list(two_df['seq1']),list(two_df['seq2'])    
regressor.evaluate_loop(10, two_0, two_1, 62, 20, "two")

three_df = pd.read_csv("three.csv")
three_0, three_1 = list(three_df['seq1']),list(three_df['seq2'])
regressor.evaluate_loop(10, three_0, three_1, 62, 20, "three")
'''
test = "AGTATTTATCAGCCCTCGCTGGATATAAGCCTGAAATCCAGGATCGCATGAAGTCCCCCCCGATACCTGGAGTGGATCCCTATGTTCGCGCAACGAACACGCGGATTTTACAGGGTGAATCACAGATT"
regressor.hash_one_seq(test, 10, 60, 20)

test2 = "AGTATTTATCAGCCCTCGCTGGATATAAGCCTGAAATCCAGGATCGCATGAAGTCCCCCCCGATACCTGGAGTGGCTCCCTATGTTCGCGCAACGAACACGCGGATTTTACAGGGTGAATCACAGATT"
regressor.hash_one_seq(test2, 10, 60, 20)

regressor.hash("loop0_info.csv", 10, 60, 20)

df = pd.read_csv("2500.csv")
#df = pd.read_csv("100thousand_upto1.csv")
sequence_pairs = [list(df['seq1']),list(df['seq2'])]

train_sequence_0, test_sequence_0, train_sequence_1, test_sequence_1 = train_test_split(sequence_pairs[0], sequence_pairs[1], test_size=0.10, shuffle=True, random_state=1)

regressor.evaluate_loop(10, test_sequence_0, test_sequence_1, 60, 20, "test")


Loaded model in model.pickle



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
100%|██████████| 1/1 [00:00<00:00, 140.28it/s]


The hashed sequence of melting temp 59.7 is ['CACCTCCCTTCTCCAGGATC']


100%|██████████| 1/1 [00:00<00:00, 159.82it/s]


The hashed sequence of melting temp 59.7 is ['TGCTGTGTGGTGGTCGTGGA']


...

Loop: On average, there are 96.472 mismatches


100%|██████████| 8/8 [00:00<00:00, 52.79it/s]


Loop: On average, there are 96.192 mismatches


100%|██████████| 8/8 [00:00<00:00, 50.71it/s]


Loop: On average, there are 96.452 mismatches
Predict: On average, there are 1.0 mismatches
Predict: On average, the melting temperatures are 59.70000000000025 and 59.70000000000025
